# Project

## 1. Libraries

In [10]:
import requests
import json
import time
import pandas as pd
from tqdm import tqdm
import psycopg2
from config import config
import configparser

# from pydantic import BaseModel, ValidationError, validator (## for the future)

## 2. Riot Games API

### i. API Parser ###

In [11]:
configParser = configparser.RawConfigParser()   
configFilePath = r'config.txt'
configParser.read(configFilePath)
api_key = configParser.get('your-config', 'api_key')

### ii. Properties ###

In [12]:
region = 'euw1'

### iii. Challenger Response (Endpoint) ###

In [13]:
#Challenger response API
challenger_url = f'https://{region}.api.riotgames.com/tft/league/v1/challenger?api_key={api_key}'
challenger_response = requests.get(url = challenger_url).json()

#Saves challenger json response as a json file 
with open("challenger_api_data.json", "w") as write_file:
    json.dump(challenger_response, write_file)

#Writes json response into a dataframe
challenger_response_df = pd.DataFrame(challenger_response['entries'])

### iv. PUUID (Endpoint) ###

In [16]:
puuid_list = []
for x in tqdm(challenger_response_df['summonerId']):
    puuid_URL = f'https://euw1.api.riotgames.com/tft/summoner/v1/summoners/{x}?api_key={api_key}'
    puuid_list.append(requests.get(url = puuid_URL).json())
    time.sleep(1.2) #100 requests every 2 mins

# #Saves puuid json response as a json file
# with open("puuid_list.json", "w") as write_file:
#     json.dump(puuid_list, write_file)

#Writes puuid json response into a dataframe
puuid_response_df = pd.DataFrame(puuid_list)
#Rename  id column in puuid_response_df to match challenger_response_df summonerId column name
puuid_response_df = puuid_response_df.rename(columns={'id' : 'summonerId'})

100%|██████████| 200/200 [04:14<00:00,  1.27s/it]


### v. Merge dataframes from different Endpoints ###

In [22]:
display(challenger_response_df)
display(puuid_response_df)

#Merges challenger df with puuid df by summonerId
merged_df = pd.merge(challenger_response_df, puuid_response_df, how='left', left_on='summonerId', right_on='summonerId')
display(merged_df)


,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,yOXZ6_7B_ASUZIodnmuPm4CVsCn_TYVC1Y9fei4dT2PIo_I,Topssen,844,I,58,370,False,False,True,False
1,jqtYuaag-L-LqML8Mf2QCiENJZPJTime4J8IpMmOZcI3lk...,Gofret TFT,980,I,31,191,False,False,False,False
2,i32g_LceMlEH-Rqqogb9coy3X0B1sae5pK15syHNbunkBV...,BBTMort,959,I,59,381,False,False,True,False
3,XPj3ftiqb6ZrhMzWdM52iS6bIDIa_I6bFyD-zhCj8f9um3M,soc un xus0o,1048,I,71,356,True,False,False,False
4,oYrDa8d8wY5pFjqjecgHMdTbAk1N5JPz9vgj8QZtljTnMG...,TFTMarx,1465,I,39,234,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...
195,-xC7Y4xTfbzoddr9X4gKZQdbfTLGMg4bosaYmJ7HbrFaMRs,Lane of Fire,953,I,59,416,True,False,False,False
196,gNkOHF9bD-79MPrHAcFsdx-_Ymcrg1N1Hk7VheQoVtrumZMS,HyunTer,1362,I,42,90,False,False,False,True
197,lIjBk2XM_p5z-64l5mXuqamYwhqbBP011hKA2RgmmYBRYRQ,CD Wavingthunder,920,I,25,169,False,False,True,False
198,xCcBif_ua4BUmRArGNezKF0aztPMOmWRzFEPUynL4pXIr56S,Moines Lewi,1094,I,35,192,False,False,False,False


,summonerId,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,yOXZ6_7B_ASUZIodnmuPm4CVsCn_TYVC1Y9fei4dT2PIo_I,uA8K7UnDtVd-sPohEbrNcxxT_39c1Y6bLdKaamNtmxtpqVE,H0mU2Fqzgw6vNr4Gcd_1cUZnAumlYYDLclzkYJv81q-j8Z...,Topssen,4022,1633272074907,383
1,jqtYuaag-L-LqML8Mf2QCiENJZPJTime4J8IpMmOZcI3lk...,0LlZyvWDBfO8MWsvlJ9k9ZOh729tAGq_SjTTvoSEhMNwYk...,ibO9vJzAktFo8KPmqKRGbl3WCdDMEAEZaCCWe0VucxWXNi...,Gofret TFT,4903,1633283566000,3
2,i32g_LceMlEH-Rqqogb9coy3X0B1sae5pK15syHNbunkBV...,GYsxwhLWMnXuaIyfIjuR7Ow2TV4EuI6NulMk6Vl4qmU_Q5...,UR8FewfA2I4So-tMB2TnLIjfNyjiZ_L6mbhLmp4Z1CFHtg...,BBTMort,5031,1633199740000,1
3,XPj3ftiqb6ZrhMzWdM52iS6bIDIa_I6bFyD-zhCj8f9um3M,th7H0pFs57izUIALLh2DuTiR_hyh_q2oCM0oMvfDB9sD4Ng,iB1bz8P7uO1EJ0M_ErPfNwALGJL9JSi7ht4XkPYfafPbaP...,soc un xus0o,5030,1633287889860,130
4,oYrDa8d8wY5pFjqjecgHMdTbAk1N5JPz9vgj8QZtljTnMG...,GqunzKooxPrSL6DTqJgZPaWrZAm2XJvRDAew3wOxs_rlvz...,5UjQjlvVTgncrqhXSJRWpsGC0VhSVE6yAH2HzSQqiJrxbR...,TFTMarx,4903,1632612911000,1
...,...,...,...,...,...,...,...
195,-xC7Y4xTfbzoddr9X4gKZQdbfTLGMg4bosaYmJ7HbrFaMRs,0loBbk0XI44M8EQAwHVM0rqbsaI3Y6MGpbqbTHeZzuzW8v4,ll0Lr8FI3beWFmDQQki_E8ghW0ldEo0T4OyTH6B8z2L94Q...,Lane of Fire,4834,1633287956000,472
196,gNkOHF9bD-79MPrHAcFsdx-_Ymcrg1N1Hk7VheQoVtrumZMS,ybuI2qpOiNN15UXwWeg2V9xOMligQizRb00mbxVzQhxQLY...,fVjAoG4qZ3cJw9wdUe2WYJzOYjgz8phSXVaHIfDc53erjr...,HyunTer,23,1633287889860,15
197,lIjBk2XM_p5z-64l5mXuqamYwhqbBP011hKA2RgmmYBRYRQ,7DnYWKyvcBlkkCOuUpxsSH_ecLMxEexvLDlHkJDQYnKn3Q,z-htx-ARhuC4qejH-HR0ETmWiQzpvtLu0zu8k6_DLnX8uV...,CD Wavingthunder,1110,1632953974000,246
198,xCcBif_ua4BUmRArGNezKF0aztPMOmWRzFEPUynL4pXIr56S,7NRUdxhaO_btQVL4twJ_VlK58VN1xkan12po1kWgbWBB06...,pVUxCF-8nA48NKQ3-jRLenMBM5SuQYSop8C0xkrVv6vmjI...,Moines Lewi,29,1633280060000,1


,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,yOXZ6_7B_ASUZIodnmuPm4CVsCn_TYVC1Y9fei4dT2PIo_I,Topssen,844,I,58,370,False,False,True,False,uA8K7UnDtVd-sPohEbrNcxxT_39c1Y6bLdKaamNtmxtpqVE,H0mU2Fqzgw6vNr4Gcd_1cUZnAumlYYDLclzkYJv81q-j8Z...,Topssen,4022,1633272074907,383
1,jqtYuaag-L-LqML8Mf2QCiENJZPJTime4J8IpMmOZcI3lk...,Gofret TFT,980,I,31,191,False,False,False,False,0LlZyvWDBfO8MWsvlJ9k9ZOh729tAGq_SjTTvoSEhMNwYk...,ibO9vJzAktFo8KPmqKRGbl3WCdDMEAEZaCCWe0VucxWXNi...,Gofret TFT,4903,1633283566000,3
2,i32g_LceMlEH-Rqqogb9coy3X0B1sae5pK15syHNbunkBV...,BBTMort,959,I,59,381,False,False,True,False,GYsxwhLWMnXuaIyfIjuR7Ow2TV4EuI6NulMk6Vl4qmU_Q5...,UR8FewfA2I4So-tMB2TnLIjfNyjiZ_L6mbhLmp4Z1CFHtg...,BBTMort,5031,1633199740000,1
3,XPj3ftiqb6ZrhMzWdM52iS6bIDIa_I6bFyD-zhCj8f9um3M,soc un xus0o,1048,I,71,356,True,False,False,False,th7H0pFs57izUIALLh2DuTiR_hyh_q2oCM0oMvfDB9sD4Ng,iB1bz8P7uO1EJ0M_ErPfNwALGJL9JSi7ht4XkPYfafPbaP...,soc un xus0o,5030,1633287889860,130
4,oYrDa8d8wY5pFjqjecgHMdTbAk1N5JPz9vgj8QZtljTnMG...,TFTMarx,1465,I,39,234,True,False,False,False,GqunzKooxPrSL6DTqJgZPaWrZAm2XJvRDAew3wOxs_rlvz...,5UjQjlvVTgncrqhXSJRWpsGC0VhSVE6yAH2HzSQqiJrxbR...,TFTMarx,4903,1632612911000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,-xC7Y4xTfbzoddr9X4gKZQdbfTLGMg4bosaYmJ7HbrFaMRs,Lane of Fire,953,I,59,416,True,False,False,False,0loBbk0XI44M8EQAwHVM0rqbsaI3Y6MGpbqbTHeZzuzW8v4,ll0Lr8FI3beWFmDQQki_E8ghW0ldEo0T4OyTH6B8z2L94Q...,Lane of Fire,4834,1633287956000,472
196,gNkOHF9bD-79MPrHAcFsdx-_Ymcrg1N1Hk7VheQoVtrumZMS,HyunTer,1362,I,42,90,False,False,False,True,ybuI2qpOiNN15UXwWeg2V9xOMligQizRb00mbxVzQhxQLY...,fVjAoG4qZ3cJw9wdUe2WYJzOYjgz8phSXVaHIfDc53erjr...,HyunTer,23,1633287889860,15
197,lIjBk2XM_p5z-64l5mXuqamYwhqbBP011hKA2RgmmYBRYRQ,CD Wavingthunder,920,I,25,169,False,False,True,False,7DnYWKyvcBlkkCOuUpxsSH_ecLMxEexvLDlHkJDQYnKn3Q,z-htx-ARhuC4qejH-HR0ETmWiQzpvtLu0zu8k6_DLnX8uV...,CD Wavingthunder,1110,1632953974000,246
198,xCcBif_ua4BUmRArGNezKF0aztPMOmWRzFEPUynL4pXIr56S,Moines Lewi,1094,I,35,192,False,False,False,False,7NRUdxhaO_btQVL4twJ_VlK58VN1xkan12po1kWgbWBB06...,pVUxCF-8nA48NKQ3-jRLenMBM5SuQYSop8C0xkrVv6vmjI...,Moines Lewi,29,1633280060000,1


In [41]:
??

Signature:
pd.merge(
    left: 'DataFrame | Series',
    right: 'DataFrame | Series',
    how: 'str' = 'inner',
    on: 'IndexLabel | None' = None,
    left_on: 'IndexLabel | None' = None,
    right_on: 'IndexLabel | None' = None,
    left_index: 'bool' = False,
    right_index: 'bool' = False,
    sort: 'bool' = False,
    suffixes: 'Suffixes' = ('_x', '_y'),
    copy: 'bool' = True,
    indicator: 'bool' = False,
    validate: 'str | None' = None,
) -> 'DataFrame'
Docstring:
Merge DataFrame or named Series objects with a database-style join.

A named Series object is treated as a DataFrame with a single named column.

The join is done on columns or indexes. If joining columns on
columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
on indexes or indexes on a column or columns, the index will be passed on.
When performing a cross merge, no column specifications to merge on are
allowed.

Parameters
----------
left : DataFrame
right : DataFrame or named Series
 

In [46]:
#  NOTES
# # entries is just a key of the json
# challenger_response.keys()
# puuid_response.keys()

dict_keys(['tier', 'leagueId', 'queue', 'name', 'entries'])

## 3. Postgresql Database

In [23]:
params = config()
# Connect to the PostgreSQL database
conn = psycopg2.connect(**params, port = 5431)
# Create a new cursor - A PostgreSQL database cursor is a read-only pointer that allows a program, regardless of the language used, to access the result set of a query. 
cur = conn.cursor()
# A function that takes in a PostgreSQL query and outputs a pandas database 

In [ ]:
# PLACEHOLDER FOR DATA VALIDATION
# class UserModel(BaseModel): for validation
#     summonerId: str
#     leaguepoints: int

In [28]:
#Loops through rows for df
for i in tqdm(range(0, merged_df.shape[0])):
    summonerName = merged_df["summonerName"][i]
    summonerId = merged_df["summonerId"][i]
    leaguePoints = merged_df["leaguePoints"][i]
    puuid = merged_df["puuid"][i]


    # do all the columns that are called TBD

    cur.execute(f'''
    INSERT INTO public."Users"(
	    id, "summonerName", "summonerId", "rankedLeague", puuid, leaguepoints, region)
	    VALUES (DEFAULT, '{summonerName}', '{summonerId}', 'CHALLENGER', '{puuid}', '{leaguePoints}', '{region}')
        ''')

    conn.commit()


100%|██████████| 200/200 [00:00<00:00, 439.24it/s]


### USEFUL TOOL - Following cell deletes existing data in database

In [26]:
#Use this cell to delete table data, although ID counter needs to be reset manually in Postgres
cur.execute(f'''
DELETE FROM public."Users"
''')

### USEFUL TOOL - Following cell manually commits a transaction to Postgres database

In [29]:
conn.commit()

## 999. Notes


### Future Development -> Data validation
If data doesn't fit an expected format, then rollback transaction from Postgres

for loop:
take API and put into SQL 
Execute 1 person at a time...
Validation of data type...

psychopg to insert data into database